In [1]:
#### Alex Weinberg
### NeoclassicalGrowthVFI
## Summer 2018

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.optimize import fminbound
from scipy import interpolate
from quantecon.markov.approximation import tauchen

#### Parameters
gamma = 0.5
beta = 0.95
delta = 0.05
alpha = 0.4

### Discretize the State space
kmin = 0
kmax = 100
ksize = 10

kgrid = np.linspace(kmin,kmax,ksize)

### Utility
def utility(c):
    ''' CRRA Utility function '''
    if c <= 0:
        c = 1e-5
    if gamma == 1:
        return np.log(c)
    else:
        return (c ** (1 - gamma)) / (1 - gamma)

def production(k,z=1):
    ''' Production function of capital and a random shock'''
    y = z * (k ** alpha)
    return y

def capital_transition(k_old,investment):
    ''' Transition equation of capital, takes in old capital and investment '''
    k_new = (1 - delta) * k_old + investment
    return k_new

# write Bellman operator function to help with VFI
def bellman_operator(Vlast):
    '''
    The approximate Bellman operator, which computes and returns the
    updated value function TV on the grid points.  An array to store
    the new set of values TV is optionally supplied (to avoid having to
    allocate new arrays at each iteration).  If supplied, any existing data in
    Tw will be overwritten.
    '''
    # Apply cubic interpolation to V
    V_func = interpolate.interp1d(kgrid, Vlast, kind='cubic', fill_value='extrapolate')

    # Initialize array for operator and policy function
    newV = np.empty_like(Vlast) # value func
    optSav = np.empty_like(newV) # savings policy

    # == set TV[i] = max_sav { u(con) + beta V(sav)} == #
    for i, k in enumerate(kgrid):
        # for each possible k_state
        def Value(sav):
            # Value today, we want to maximize this
            return - utility(k - sav) - beta * V_func(sav)
        # below returns root, optimal savings (max of the Value today)
        sav_star = fminbound(Value, 1e-6, k - 1e-6)
        optSav[i] = sav_star
        newV[i] = - Value(sav_star)
    return newV, optSav


# Options
tol = 1e-5
maxiter = 100
diff = 1
its = 0

Vguess = np.zeros(ksize)
V = Vguess
sav = np.zeros(ksize)
savind = np.zeros(ksize)
con = np.zeros(ksize)

while diff < tol and its > maxiter:
    Vlast = V
    V = np.zeros_like(Vlast)

    V, optSav = bellman_operator(Vlast)
    diff = (np.absolute(V - Vlast)).max()
    
    print('Iteration ', its, ', distance = ', diff)
    its += 1

